In [1]:
from tensorflow.keras.models import load_model
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalAveragePooling1D, Dense
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

In [2]:
# Load the dataset
data = pd.read_csv("25.csv")

# Separate data into different dataframes based on data types
spo2_data = data[data['data_type'] == 'spo2'].copy()
heart_rate_data = data[data['data_type'] == 'hr'].copy()
steps_data = data[data['data_type'] == 'steps'].copy()
hrv_data = data[data['data_type'] == 'hrv'].copy()
br_data = data[data['data_type'] == 'br'].copy()
sleep_data = data[data['data_type'] == 'sleep'].copy()

# Convert timestamps to datetime format for each dataframe
spo2_data['time'] = pd.to_datetime(spo2_data['time'])
heart_rate_data['time'] = pd.to_datetime(heart_rate_data['time'])
steps_data['time'] = pd.to_datetime(steps_data['time'])
hrv_data['time'] = pd.to_datetime(hrv_data['time'])
br_data['time'] = pd.to_datetime(br_data['time'])
sleep_data['time'] = pd.to_datetime(sleep_data['time'])

# Sort dataframes based on time and their respective formats
spo2_data.sort_values(by='time', inplace=True)
heart_rate_data.sort_values(by='time', inplace=True)
steps_data.sort_values(by='time', inplace=True)
hrv_data.sort_values(by='time', inplace=True)
br_data.sort_values(by='time', inplace=True)
sleep_data.sort_values(by='time', inplace=True)

In [3]:
heart_rate_data=heart_rate_data[['value', 'time']]

In [4]:
heart_rate_data

,value,time
1,64,2024-03-08 00:00:00+00:00
3,62,2024-03-08 00:01:00+00:00
4,63,2024-03-08 00:02:00+00:00
5,65,2024-03-08 00:03:00+00:00
6,65,2024-03-08 00:04:00+00:00
...,...,...
40503,73,2024-04-05 23:55:00+00:00
40504,73,2024-04-05 23:56:00+00:00
40505,74,2024-04-05 23:57:00+00:00
40506,74,2024-04-05 23:58:00+00:00


In [5]:
steps_data=steps_data[['value', 'time']]

In [6]:
steps_data

,value,time
46,15,2024-03-08 01:11:00+00:00
48,8,2024-03-08 01:13:00+00:00
85,25,2024-03-08 01:49:00+00:00
116,9,2024-03-08 02:17:00+00:00
130,8,2024-03-08 02:29:00+00:00
...,...,...
40402,7,2024-04-05 22:18:00+00:00
40412,7,2024-04-05 22:28:00+00:00
40428,13,2024-04-05 22:42:00+00:00
40450,4,2024-04-05 23:03:00+00:00


In [7]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Load craving and non-craving data
craving_data = pd.read_csv('ID25_Crave.csv')
non_craving_data = pd.read_csv('ID25_None.csv')

# Filter rows with 'Carrot' in the substance_fruit_label column
craving_data = craving_data[craving_data['substance_fruit_label'] == 'Carrot']

# Convert timestamps to ensure they are timezone-naive
craving_data['hawaii_createdat_time'] = pd.to_datetime(craving_data['hawaii_createdat_time']).dt.tz_localize(None)
non_craving_data['hawaii_createdat_time'] = pd.to_datetime(non_craving_data['hawaii_createdat_time']).dt.tz_localize(None)

# Ensure heart rate and steps data use the same timezone-naive datetime index
heart_rate_data['time'] = pd.to_datetime(heart_rate_data['time']).dt.tz_localize(None)
steps_data['time'] = pd.to_datetime(steps_data['time']).dt.tz_localize(None)

heart_rate_data.set_index('time', inplace=True)
steps_data.set_index('time', inplace=True)

# Convert the 'value' column to numeric, coercing errors
heart_rate_data['value'] = pd.to_numeric(heart_rate_data['value'], errors='coerce')
steps_data['value'] = pd.to_numeric(steps_data['value'], errors='coerce')

# Remove duplicate timestamps
heart_rate_data = heart_rate_data[~heart_rate_data.index.duplicated(keep='first')]
steps_data = steps_data[~steps_data.index.duplicated(keep='first')]

# Reindex and forward fill to handle missing data
heart_rate_data = heart_rate_data.reindex(pd.date_range(start=heart_rate_data.index.min(), end=heart_rate_data.index.max(), freq='min'), method='ffill')
steps_data = steps_data.reindex(pd.date_range(start=steps_data.index.min(), end=steps_data.index.max(), freq='min'), method='ffill').fillna(0)

# Initialize the StandardScaler
scaler = StandardScaler()

def process_label_data(label_data, heart_rate_data, steps_data, scaler):
    results = []
    for _, row in label_data.iterrows():
        timestamp = row['hawaii_createdat_time']
        lower_bound = timestamp - pd.Timedelta(hours=1)
        upper_bound = timestamp + pd.Timedelta(hours=1)

        # Extract data within the window
        hr_window = heart_rate_data.loc[lower_bound:upper_bound]
        steps_window = steps_data.loc[lower_bound:upper_bound]

        # Resample and calculate mean every 4 minutes to get 30 points
        hr_means = hr_window.resample('4min').mean().iloc[:30]  # Ensure 30 data points
        steps_means = steps_window.resample('4min').mean().iloc[:30]

        if len(hr_means) == 30 and len(steps_means) == 30:
            # Standardize the means
            hr_scaled = scaler.fit_transform(hr_means.values.reshape(-1, 1))
            steps_scaled = scaler.fit_transform(steps_means.values.reshape(-1, 1))

            results.append({
                'timestamp': timestamp,
                'heart_rate_means': hr_scaled.flatten().tolist(),
                'steps_means': steps_scaled.flatten().tolist()
            })

    return pd.DataFrame(results)

# Process craving and non-craving labels
craving_features = process_label_data(craving_data, heart_rate_data, steps_data, scaler)
non_craving_features = process_label_data(non_craving_data, heart_rate_data, steps_data, scaler)

# Output results
print("Craving Features Data:")
print(craving_features.head())
print("Non-Craving Features Data:")
print(non_craving_features.head())


Craving Features Data:
            timestamp                                   heart_rate_means  \
0 2024-03-08 21:00:35  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
1 2024-03-09 21:01:05  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
2 2024-03-11 21:03:19  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
3 2024-03-21 15:20:07  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
4 2024-03-22 10:00:44  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   

                                         steps_means  
0  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
1  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
2  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
3  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
4  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...  
Non-Craving Features Data:
            timestamp                                   heart_rate_means  \
0 2024-03-08 10:41:54  [-0.09184025046275009, 0.2323018099940171, 0.3...   
1 2024-03-0

In [8]:
# Print the shape of the craving features data
print("Shape of Craving Features Data:", craving_features.shape)

# Print the shape of the non-craving features data
print("Shape of Non-Craving Features Data:", non_craving_features.shape)

Shape of Craving Features Data: (5, 3)
Shape of Non-Craving Features Data: (91, 3)


In [9]:
import pandas as pd

# Adding label columns
craving_features['state'] = 'crave'
craving_features['state_val'] = 1

non_craving_features['state'] = 'non_crave'
non_craving_features['state_val'] = 0

# Combining the dataframes
combined_data = pd.concat([craving_features, non_craving_features])

# Converting timestamps to datetime and sorting
combined_data['timestamp'] = pd.to_datetime(combined_data['timestamp'])
combined_data_sorted = combined_data.sort_values(by='timestamp').reset_index(drop=True)

# This combined_data_sorted is now ready for use in your neural network model
print(combined_data_sorted)


             timestamp                                   heart_rate_means  \
0  2024-03-08 10:41:54  [-0.09184025046275009, 0.2323018099940171, 0.3...   
1  2024-03-08 13:00:44  [0.6393592140252059, -0.21638821778266962, -0....   
2  2024-03-08 17:03:27  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
3  2024-03-08 21:00:35  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
4  2024-03-09 10:25:48  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...   
..                 ...                                                ...   
91 2024-04-04 13:14:39  [-0.7126238512538401, -0.3477993540249117, -0....   
92 2024-04-04 21:00:13  [-1.1404840042919875, -1.325164652714269, -1.4...   
93 2024-04-05 09:16:50  [-1.3279360019674473, -2.0010052538412895, -1....   
94 2024-04-05 13:13:37  [-1.5838886684145206, -1.6526537336134144, -1....   
95 2024-04-05 21:00:17  [3.3461584365958, 1.806781013280667, 0.8311192...   

                                          steps_means      state  state_val

In [10]:
import tensorflow as tf

# Load pretrained models
encoder_hr = tf.keras.models.load_model('heart_rate_encoder')
encoder_steps = tf.keras.models.load_model('steps_encoder')

In [11]:
import numpy as np

# Function to generate embeddings using the loaded models
def generate_embeddings(hr_data, steps_data, encoder_hr, encoder_steps):
    # Convert lists to numpy arrays and ensure they are in the correct shape
    hr_data = np.array(hr_data.tolist())
    steps_data = np.array(steps_data.tolist())

    # Reshape data if required by the model, assuming the model expects shape (samples, features)
    if hr_data.ndim == 1:
        hr_data = hr_data.reshape(-1, 1)
    if steps_data.ndim == 1:
        steps_data = steps_data.reshape(-1, 1)

    # Generate embeddings
    hr_embeddings = encoder_hr.predict(hr_data)
    steps_embeddings = encoder_steps.predict(steps_data)

    return hr_embeddings, steps_embeddings

# Use the function with your data
hr_embeddings, steps_embeddings = generate_embeddings(
    combined_data_sorted['heart_rate_means'],
    combined_data_sorted['steps_means'],
    encoder_hr,
    encoder_steps
)


3/3 [==============================] - 0s 3ms/step


In [12]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve, auc
from sklearn.metrics import recall_score, precision_score, confusion_matrix, classification_report
from sklearn.utils.class_weight import compute_class_weight

# Combine the heart rate and steps embeddings
combined_embeddings = np.concatenate([hr_embeddings, steps_embeddings], axis=1)

# Labels from your existing DataFrame
labels = combined_data_sorted['state_val'].values

# Calculate class weights
class_weights = compute_class_weight('balanced', classes=np.unique(labels), y=labels)
class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(combined_embeddings, labels, test_size=0.2, random_state=42)

# Define the model with regularization and batch normalization
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],), kernel_regularizer=l2(0.01)),
    BatchNormalization(),
    Dropout(0.5),
    Dense(32, activation='relu', kernel_regularizer=l2(0.01)),
    Dropout(0.5),
    Dense(16, activation='relu', kernel_regularizer=l2(0.01)),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy', Precision(), Recall()])

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=16,
    validation_split=0.1,
    class_weight=class_weight_dict,
    callbacks=[early_stopping],
    verbose=1
)

# Evaluate thresholds on training set
probabilities_train = model.predict(X_train)
thresholds = np.arange(0, 1.01, 0.01)

# Store training results
training_results = []

print("\nTraining Set Evaluation:")
for threshold in thresholds:
    predicted_classes = (probabilities_train > threshold).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_train, predicted_classes).ravel()
    recall = tp / (tp + fn)  # Sensitivity
    specificity = tn / (tn + fp)  # Specificity
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    training_results.append((threshold, recall, specificity, accuracy))
    print(f"Threshold: {threshold:.2f}, Sensitivity: {recall:.2f}, Specificity: {specificity:.2f}, Accuracy: {accuracy:.2f}")

# Convert training results to DataFrame for easy viewing
training_results_df = pd.DataFrame(training_results, columns=['Threshold', 'Sensitivity', 'Specificity', 'Accuracy'])

# Select best threshold based on a specific criterion
# Here, we select the threshold with the highest sensitivity while keeping specificity above 0.5
best_threshold_row = training_results_df[(training_results_df['Sensitivity'] > 0.9) & (training_results_df['Specificity'] > 0.5)]
if not best_threshold_row.empty:
    best_threshold = best_threshold_row.iloc[0]['Threshold']
else:
    best_threshold = 0.5  # Default threshold if criteria is not met

print(f"\nBest Threshold from Training Set: {best_threshold}")

# Evaluate the best threshold on the test set
probabilities_test = model.predict(X_test)
predicted_classes_test = (probabilities_test > best_threshold).astype(int)
tn, fp, fn, tp = confusion_matrix(y_test, predicted_classes_test).ravel()
recall_test = tp / (tp + fn)  # Sensitivity
specificity_test = tn / (tn + fp)  # Specificity
accuracy_test = (tp + tn) / (tp + tn + fp + fn)

print(f"\nTest Set Evaluation at Best Threshold ({best_threshold}):")
print(f"Sensitivity: {recall_test:.2f}")
print(f"Specificity: {specificity_test:.2f}")
print(f"Accuracy: {accuracy_test:.2f}")

# Store test results
test_results = []

for threshold in thresholds:
    predicted_classes = (probabilities_test > threshold).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_test, predicted_classes).ravel()
    recall = tp / (tp + fn)  # Sensitivity
    specificity = tn / (tn + fp)  # Specificity
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    test_results.append((threshold, recall, specificity, accuracy))
    print(f"Threshold: {threshold:.2f}, Sensitivity: {recall:.2f}, Specificity: {specificity:.2f}, Accuracy: {accuracy:.2f}")

# Convert test results to DataFrame for easy viewing
test_results_df = pd.DataFrame(test_results, columns=['Threshold', 'Sensitivity', 'Specificity', 'Accuracy'])

# Find the threshold where specificity is around 0.9 and sensitivity is around 0.5
desired_specificity = 0.9
desired_sensitivity_range = (0.4, 0.6)  # Considering sensitivity around 0.5

filtered_results = test_results_df[
    (test_results_df['Specificity'] >= desired_specificity) & 
    (test_results_df['Sensitivity'] >= desired_sensitivity_range[0]) & 
    (test_results_df['Sensitivity'] <= desired_sensitivity_range[1])
]

# Print the filtered results
print("\nFiltered Thresholds with Specificity >= 0.9 and Sensitivity ~ 0.5:")
print(filtered_results)

# Plot the results for better visualization
plt.figure(figsize=(12, 6))
plt.plot(test_results_df['Threshold'], test_results_df['Sensitivity'], label='Sensitivity')
plt.plot(test_results_df['Threshold'], test_results_df['Specificity'], label='Specificity')
plt.plot(test_results_df['Threshold'], test_results_df['Accuracy'], label='Accuracy')
plt.axhline(y=0.9, color='r', linestyle='--', label='Desired Specificity (0.9)')
plt.axhline(y=0.5, color='g', linestyle='--', label='Desired Sensitivity (0.5)')
plt.xlabel('Threshold')
plt.ylabel('Score')
plt.title('Sensitivity, Specificity, and Accuracy vs. Threshold')
plt.legend()
plt.grid(True)
plt.show()

Epoch 1/50


5/5 [==============================] - 3s 119ms/step - loss: 2.6216 - accuracy: 0.8676 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 2.2546 - val_accuracy: 0.7500 - val_precision: 0.3333 - val_recall: 1.0000
Epoch 2/50
5/5 [==============================] - 0s 16ms/step - loss: 2.7178 - accuracy: 0.8235 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 2.1815 - val_accuracy: 0.7500 - val_precision: 0.3333 - val_recall: 1.0000
Epoch 3/50
5/5 [==============================] - 0s 14ms/step - loss: 2.0999 - accuracy: 0.7647 - precision: 0.1250 - recall: 0.5000 - val_loss: 2.1147 - val_accuracy: 0.6250 - val_precision: 0.2500 - val_recall: 1.0000
Epoch 4/50
5/5 [==============================] - 0s 16ms/step - loss: 1.9782 - accuracy: 0.8971 - precision: 0.2857 - recall: 0.5000 - val_loss: 2.0545 - val_accuracy: 0.6250 - val_precision: 0.2500 - val_recall: 1.0000
Epoch 5/50
5/5 [==============================] - 0s 17ms/step - loss: 2.2673 - accuracy: 0.8824 

5/5 [==============================] - 0s 15ms/step - loss: 1.2664 - accuracy: 0.8382 - precision: 0.1818 - recall: 0.5000 - val_loss: 1.3697 - val_accuracy: 0.6250 - val_precision: 0.2500 - val_recall: 1.0000
Epoch 36/50
5/5 [==============================] - 0s 15ms/step - loss: 1.3208 - accuracy: 0.8235 - precision: 0.1667 - recall: 0.5000 - val_loss: 1.3595 - val_accuracy: 0.6250 - val_precision: 0.2500 - val_recall: 1.0000
Epoch 37/50
5/5 [==============================] - 0s 15ms/step - loss: 1.0885 - accuracy: 0.8824 - precision: 0.3333 - recall: 1.0000 - val_loss: 1.3480 - val_accuracy: 0.6250 - val_precision: 0.2500 - val_recall: 1.0000
Epoch 38/50
5/5 [==============================] - 0s 14ms/step - loss: 1.2571 - accuracy: 0.8676 - precision: 0.3077 - recall: 1.0000 - val_loss: 1.3377 - val_accuracy: 0.6250 - val_precision: 0.2500 - val_recall: 1.0000
Epoch 39/50
5/5 [==============================] - 0s 15ms/step - loss: 1.3131 - accuracy: 0.8235 - precision: 0.1667 - reca

1/1 [==============================] - 0s 27ms/step

Test Set Evaluation at Best Threshold (0.39):
Sensitivity: nan
Specificity: 0.40
Accuracy: 0.40
Threshold: 0.00, Sensitivity: nan, Specificity: 0.00, Accuracy: 0.00
Threshold: 0.01, Sensitivity: nan, Specificity: 0.00, Accuracy: 0.00
Threshold: 0.02, Sensitivity: nan, Specificity: 0.00, Accuracy: 0.00
Threshold: 0.03, Sensitivity: nan, Specificity: 0.00, Accuracy: 0.00
Threshold: 0.04, Sensitivity: nan, Specificity: 0.00, Accuracy: 0.00
Threshold: 0.05, Sensitivity: nan, Specificity: 0.00, Accuracy: 0.00
Threshold: 0.06, Sensitivity: nan, Specificity: 0.00, Accuracy: 0.00
Threshold: 0.07, Sensitivity: nan, Specificity: 0.00, Accuracy: 0.00
Threshold: 0.08, Sensitivity: nan, Specificity: 0.00, Accuracy: 0.00
Threshold: 0.09, Sensitivity: nan, Specificity: 0.00, Accuracy: 0.00
Threshold: 0.10, Sensitivity: nan, Specificity: 0.00, Accuracy: 0.00
Threshold: 0.11, Sensitivity: nan, Specificity: 0.00, Accuracy: 0.00
Threshold: 0.12, Sensit

C:\Users\Ali\AppData\Local\Temp\ipykernel_20668\682625456.py:90: RuntimeWarning: invalid value encountered in scalar divide
  recall_test = tp / (tp + fn)  # Sensitivity
C:\Users\Ali\AppData\Local\Temp\ipykernel_20668\682625456.py:105: RuntimeWarning: invalid value encountered in scalar divide
  recall = tp / (tp + fn)  # Sensitivity
C:\Users\Ali\AppData\Local\Temp\ipykernel_20668\682625456.py:105: RuntimeWarning: invalid value encountered in scalar divide
  recall = tp / (tp + fn)  # Sensitivity
C:\Users\Ali\AppData\Local\Temp\ipykernel_20668\682625456.py:105: RuntimeWarning: invalid value encountered in scalar divide
  recall = tp / (tp + fn)  # Sensitivity
C:\Users\Ali\AppData\Local\Temp\ipykernel_20668\682625456.py:105: RuntimeWarning: invalid value encountered in scalar divide
  recall = tp / (tp + fn)  # Sensitivity
C:\Users\Ali\AppData\Local\Temp\ipykernel_20668\682625456.py:105: RuntimeWarning: invalid value encountered in scalar divide
  recall = tp / (tp + fn)  # Sensitivity


ValueError: not enough values to unpack (expected 4, got 1)

In [ ]:
Threshold: 0.48, Sensitivity: 0.58, Specificity: 0.78, Accuracy: 0.67
Threshold: 0.48, Sensitivity: 0.75, Specificity: 1.00, Accuracy: 0.83